Import libs

In [26]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
x,y = sp.symbols('x y') # Инициализириуем переменные
sp.init_printing()  
f = x**2 + y**2 #задаем функцию

Define gradient func

In [28]:
def gradient(f):
    symbolic_gradient = (f.diff(x), f.diff(y))

    print(symbolic_gradient)

    x1 = sp.lambdify(((x, y)),symbolic_gradient[0])
    x2 = sp.lambdify(((x, y)),symbolic_gradient[1])

    grad = lambda x,y: [x1(x,y), x2(x,y)]
    return grad


Define Gradient Descent

In [44]:
def grad_descent(f, grad_f, x0: np.array, lr: float = 0.1, eps: float = 1e-06):
    func = sp.lambdify(((x, y)),f) # инициализируем значение z функции 
    pointFirst = np.append(x0, func(x0[0], x0[1])) # Задаем первую точку
    res = [pointFirst] # Закидываем первую точку в результат
    while True:      
        x0 = x0 - lr*np.array(grad_f(x0[0], x0[1])) # Формула классического градиентного спуска
        
        point = np.append(x0, func(x0[0], x0[1]))
        res.append(point)
        norm = np.linalg.norm(np.array(grad_f(x0[0], x0[1]))) # норма вектора(приведение 3х мерного вектора в 1мерный с целью сравнить с eps)
        print("n = {:4d} || f = {:.4e} || lr = {:.2e} || norm = {:.2e}".format(len(res) - 1, res[-1][-1], lr, norm)) # Debug значений
        
        if norm < eps:
            break
    return np.array(res)

Define Adagrad Descent


In [30]:
def adagrad_descent(f, grad_f, x0: np.ndarray, lr: float = 0.1, eps: float = 1e-06, eps_ada : float = 1e-06):
    func = sp.lambdify(((x, y)),f)
    
    pointFirst = np.append(x0, func(x0[0], x0[1]))

    res = [pointFirst]
    Y = []
    G = 0
    while True:
        grad = np.array(grad_f(x0[0], x0[1])) #Градиент в точке
        G += grad**2 # формула адаграда
        x0 = x0 - (lr*grad)/np.sqrt(G+eps_ada)

        point = np.append(x0, func(x0[0], x0[1]))
        res.append(point)     
        norm = np.linalg.norm(grad)
        print("n = {:4d} || f = {:.4e} || lr = {:.2e} || norm = {:.2e}".format(len(res) - 1, res[-1][-1], lr, norm))
        
        if norm < eps:
            break
    return np.array(res)

Define Momentum Descent


In [31]:
def momentum_descent(f, grad_f, x0: np.ndarray, lr: float = 0.1, eps: float = 1e-06, beta : float = 0.9):
    func = sp.lambdify(((x, y)),f)
    
    pointFirst = np.append(x0, func(x0[0], x0[1]))

    res = [pointFirst]
    Y = []
    m = np.zeros_like(x0)
    while True:
        grad = np.array(grad_f(x0[0], x0[1]))
        
        #формула моментого метода
        m = beta * m - lr * grad
        x0 = x0 + m

        point = np.append(x0, func(x0[0], x0[1]))
        res.append(point)
        
        norm = np.linalg.norm(grad)
        print("n = {:4d} || f = {:.4e} || lr = {:.2e} || norm = {:.2e}".format(len(res) - 1, res[-1][-1], lr, norm))
        
        if norm < eps:
            break
    return np.array(res)

Setting Plot 

In [32]:
def column(matrix, i):
    return [row[i] for row in matrix]

def show_graph(f,points):

    last_p_x = column(points, 0)[-1]
    last_p_y = column(points, 1)[-1]

    funca = sp.lambdify(((x, y)),f)
   
    x_v = np.linspace(last_p_x-100, x0[0], 100) 
    y_v = np.linspace(last_p_y-100, x0[1], 100)
   
    X, Y = np.meshgrid(x_v, y_v)
    Z = funca(X, Y)

    #Задаем пространство график f(параболоид)
    fig = go.Figure()
    fig.add_surface(x=X, y=Y, z=Z, colorscale='aggrnyl',opacity = 0.7)

    #Задаем траекторию градиентного спуска
    line_marker = dict(color='red', width=7)
    fig.add_scatter3d(x=column(points, 0), y=column(points, 1), z=column(points, 2), mode='lines', line=line_marker, name='')


    fig.update_layout(title='Grad descent', autosize=False,
                    width=500, height=500,
                    margin=dict(l=20, r=20, b=15, t=40))


    fig.show() 
    return

Gradient Descent(standart) run

In [45]:
#задаем начальное значение
x0 = np.array([100,100])

#Вызывыем функцию градиентного спуска с заданными параметрами
points = grad_descent(f, gradient(f), x0,lr=1e-1)

#Выводим крайнюю точку(минимум(первую с конца))
print(points[-1])

#Вызываем функцию отрисовки графика с заданными (f(график функции), points(график спуска)
show_graph(f, points)



(2*x, 2*y)
n =    1 || f = 1.2800e+04 || lr = 1.00e-01 || norm = 2.26e+02
n =    2 || f = 8.1920e+03 || lr = 1.00e-01 || norm = 1.81e+02
n =    3 || f = 5.2429e+03 || lr = 1.00e-01 || norm = 1.45e+02
n =    4 || f = 3.3554e+03 || lr = 1.00e-01 || norm = 1.16e+02
n =    5 || f = 2.1475e+03 || lr = 1.00e-01 || norm = 9.27e+01
n =    6 || f = 1.3744e+03 || lr = 1.00e-01 || norm = 7.41e+01
n =    7 || f = 8.7961e+02 || lr = 1.00e-01 || norm = 5.93e+01
n =    8 || f = 5.6295e+02 || lr = 1.00e-01 || norm = 4.75e+01
n =    9 || f = 3.6029e+02 || lr = 1.00e-01 || norm = 3.80e+01
n =   10 || f = 2.3058e+02 || lr = 1.00e-01 || norm = 3.04e+01
n =   11 || f = 1.4757e+02 || lr = 1.00e-01 || norm = 2.43e+01
n =   12 || f = 9.4447e+01 || lr = 1.00e-01 || norm = 1.94e+01
n =   13 || f = 6.0446e+01 || lr = 1.00e-01 || norm = 1.55e+01
n =   14 || f = 3.8686e+01 || lr = 1.00e-01 || norm = 1.24e+01
n =   15 || f = 2.4759e+01 || lr = 1.00e-01 || norm = 9.95e+00
n =   16 || f = 1.5846e+01 || lr = 1.00e-01 

Adagrad Descent run

In [ ]:
#задаем начальное значение
x0 = np.array([100.0,100.0])

points = adagrad_descent(f, gradient(f), x0, lr=1e1)
 
print(points[-1])

show_graph(f, points)


Momentum Descent run

In [ ]:

x0 = np.array([100.0,100.0])

points = momentum_descent(f, gradient(f), x0, lr=1)
 
print(points[-1])

show_graph(f, points)